---
title: "A step by step guide to build the HELLO dialect for MLIR"
description: "Building a MLIR dialect from 0"
author: "Victor Guerra"
date: 2023-05-16
date-modified: last-modified
date-format: full
draft: true
page-layout: full
categories: 
    - mlir
    - compilers
    - posts
---

In this series of posts, we are going to see how to build from scratch a MLIR dialect. The dialect we are building here is the [*MLIR Hello Dialect*](https://github.com/Lewuathe/mlir-hello) by [Kai Sasaki](https://www.lewuathe.com/).

## Prerequisites

Throught the different posts I am going to assume you have a local build of MLIR. If you don't, you can checkout the [official getting started guide](https://mlir.llvm.org/getting_started/) for MLIR.

## Overview of the Hello dialect

The hello dialect provides 3 basic operations:

* `hello.constant`: Constant operation that turns a literal into a SSA value.
* `hello.print`: You guessed it, a print operation.
* `hello.world`: Operation that prints "Hello, World".

For a more detialed description of the operations checkout the [README of the Hello Dialect repo](https://github.com/Lewuathe/mlir-hello#operations).

## Setting up the build system

We start by setting up CMake with a minimal set of commands to build our project, for that we need a `CMakeList.txt` file:

```cmake
cmake_minimum_required(VERSION 3.20.0)

project(mlir-hello LANGUAGES CXX C)

set(CMAKE_CXX_STANDARD 17 CACHE STRING "C++ standard to conform to")

find_package(MLIR REQUIRED CONFIG)

message(STATUS "Using MLIRConfig.cmake in: ${MLIR_DIR}")
message(STATUS "Using LLVMConfig.cmake in: ${LLVM_DIR}")

```

First, we name the project `mlir-hello` and specified the languages needed to build it. `C++` and `C` in this case. Then we make sure that we locate the `MLIR` package (and `LLVM` subsequentially) and load its specific details. This is the most basic `CMake` setup that you need to start working on your own dialect. You can already try to build your project using this `bash` script:


```bash
rm -rf build
mkdir build


pushd build

LLVM_BUILD_DIR=<PATH TO LLVMs BUILD DIRECTORY>

cmake -G Ninja .. \
    -DLLVM_DIR="$LLVM_BUILD_DIR/lib/cmake/llvm" \
    -DMLIR_DIR="$LLVM_BUILD_DIR/lib/cmake/mlir" \
    -DCMAKE_BUILD_TYPE=Debug

cmake --build .

popd
```

Note that `LLVM_BUILD_DIR` should point to the `build` directory of your local MLIR build.

## Defining the dialect

Now we move forward to define the Hello Dialect in the [ODS](https://mlir.llvm.org/docs/DefiningDialects/Operations/) format.

```tablegen
#ifndef HELLO_DIALECT
#define HELLO_DIALECT

include "mlir/IR/OpBase.td"

//===----------------------------------------------------------------------===//
// Hello dialect definition.
//===----------------------------------------------------------------------===//

def Hello_Dialect : Dialect {
    let name = "hello";
    let summary = "A hello out-of-tree MLIR dialect.";
    let description = [{
        This dialect is minimal example to implement hello-world kind of sample code
        for MLIR.
    }];
    let cppNamespace = "::hello";
    let hasConstantMaterializer = 1;
}

#endif // HELLO_DIALECT
```

Let's name this file `HelloDialect.td`. But, where do we place it? Following the recommendation of [`Creating a Dialect`](https://mlir.llvm.org/docs/Tutorials/CreatingADialect/) tutorial, we should place the `TableGen` files in the `include` directory. So let's create the following layout of directories:

```bash
./
├── CMakeLists.txt
└── include/
    ├── CMakeLists.txt
    └── Hello/
        ├── CMakeLists.txt
        └── HelloDialect.td
```

Let's have a look at what the newly introduced `CMakeLists.txt` files should be.

`./include/CMakeLists.txt` is simple, we just need to signal the existance of the `Hello` directory:

```cmake
add_subdirectory(Hello)
```

`./include/Hello/CMakeLists.txt` contains some `MLIR` related commands:

```cmake
add_mlir_dialect(HelloDialect hello)
add_mlir_doc(HelloDialect HelloDialect Hello/ -gen-dialect-doc)
```

`add_mlir_dialect` declares a dialect in the include directory. Its parameters are the dialect name and the namespace to be used.

`add_mlir_doc` generates the documenation for the dialect. Its parameters are the td filename, the name of the output markdown file, the name of the directory where to put the documentation and finally the command to execute.